In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
raw_df = keras.datasets.fashion_mnist

In [4]:
raw_df

<module 'keras.api._v2.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.7/dist-packages/keras/api/_v2/keras/datasets/fashion_mnist/__init__.py'>

In [5]:
(train_images, train_labels), (test_images, test_labels) = raw_df.load_data()

In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [9]:
train_images[0].shape

(28, 28, 1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [12]:
tuner_search = RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5, directory = 'output', project_name = 'FashionMNIST')

INFO:tensorflow:Reloading Oracle from existing project output/FashionMNIST/oracle.json
INFO:tensorflow:Reloading Tuner from output/FashionMNIST/tuner0.json


In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [15]:
model = tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 128)       358528    
                                                                 
 flatten (Flatten)           (None, 61952)             0         
                                                                 
 dense (Dense)               (None, 64)                3964992   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 4,325,290
Trainable params: 4,325,290
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 14s 7ms/step - loss: 0.2856 - accuracy: 0.8952 - val_loss: 0.3624 - val_accuracy: 0.8693
Epoch 5/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2698 - accuracy: 0.9001 - val_loss: 0.3551 - val_accuracy: 0.8752
Epoch 6/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2518 - accuracy: 0.9076 - val_loss: 0.3808 - val_accuracy: 0.8730
Epoch 7/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2326 - accuracy: 0.9144 - val_loss: 0.4470 - val_accuracy: 0.8767
Epoch 8/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2182 - accuracy: 0.9198 - val_loss: 0.4242 - val_accuracy: 0.8757
Epoch 9/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2089 - accuracy: 0.9239 - val_loss: 0.4702 - val_accuracy: 0.8757
Epoch 10/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2020 - accuracy: 0.9288 - val_loss: 0.4633 - val_a